In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras import optimizers
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def loadData(path):
  data = pd.read_csv(path)
  data = data.dropna()
  print("Dimensions of dataset:", data.shape)
  print(data.columns)
  print(data.head)
  return data

In [4]:
#Load training data
print("Loading Train data:")
trainData = loadData("/content/drive/My Drive/Train_Glove_Embeddings.csv")

#Load test data
print("Loading Test data:")
testData = loadData("/content/drive/My Drive/Test_Glove_Embeddings.csv")

Loading Train data:
Dimensions of dataset: (159468, 309)
Index(['Unnamed: 0', 'id', 'comment_text', '0', '1', '2', '3', '4', '5', '6',
       ...
       '296', '297', '298', '299', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate'],
      dtype='object', length=309)
<bound method NDFrame.head of         Unnamed: 0                id  ... insult  identity_hate
0                0  0000997932d777bf  ...    0.0            0.0
1                1  000103f0d9cfb60f  ...    0.0            0.0
2                2  000113f07ec002fd  ...    0.0            0.0
3                3  0001b41b1c6bb37e  ...    0.0            0.0
4                4  0001d958c54c6e35  ...    0.0            0.0
...            ...               ...  ...    ...            ...
159546      159546  ffb47123b2d82762  ...    1.0            0.0
159547      159547  ffb7b4c3d3ae5842  ...    0.0            0.0
159548      159548  ffb93b0a0a1e78f9  ...    0.0            0.0
159549      159549  ffb998f9749bd8

In [0]:
def initilializeCNN(noOfFilters, kernelSize, dropOut):
  model = Sequential()
  model.add(Conv1D(filters = noOfFilters, kernel_size = kernelSize, activation='relu', input_shape=(300, 1)))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(noOfFilters, activation='relu', kernel_initializer = "random_uniform"))
  model.add(Dropout(dropOut))
  model.add(Dense(6, activation='softmax'))
  rmsprop = optimizers.RMSprop(lr = 0.001)
  model.compile(loss='categorical_crossentropy', optimizer = rmsprop, metrics=['accuracy'])

  return model

In [6]:
#Set model parameters
model = initilializeCNN(64, 3, 0.2)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [7]:
%%capture

#Train model on training data
trainX = np.array(trainData[trainData.columns[3:303]])
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], 1)
print("Training data input to model:", trainX.shape)

trainY = trainData[trainData.columns[303:]]
# trainY = trainY.replace(0, 1e-10)
trainY = np.array(trainY)
print("Training labels input to model:", trainY.shape)
print(trainY)
epochs = 2
batch_size = 32

model.fit(trainX, trainY, epochs=epochs, validation_split=0.1, batch_size = batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [0]:
def predict(model, threshold, testX):
  predictions = model.predict(testX)
  for i in range(predictions.shape[0]):
    for j in range(predictions.shape[1]):
      if predictions[i][j] < 0.5:
        predictions[i][j] = 0
      else:
        predictions[i][j] = 1
  return predictions

In [9]:
#Evaluating on test data

testX = np.array(testData[trainData.columns[3:303]])
testX = testX.reshape(testX.shape[0], testX.shape[1], 1)
print("Test data dimensions to model:", testX.shape)

testY = testData[testData.columns[303:]]
# testY = testY.replace(0, 1e-10)
testY = np.array(testY)
print("Test features dimensions to model:", testY.shape)
print("Features:", testY)

predictions = predict(model, 0.5, testX)
# print(predictions.shape)
print("predictions:", predictions)

Test data dimensions to model: (63380, 300, 1)
Test features dimensions to model: (63380, 6)
Features: [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [1 0 1 0 1 0]
 [0 0 0 0 0 0]]
predictions: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [11]:
#calculate score
loss = log_loss(testY,predictions)
print("Log_loss : {}".format(loss))
predictions = np.round(predictions)
loss = hamming_loss(testY,predictions)
print("Hamming_loss : {}".format(loss*100))
accuracy = accuracy_score(testY,predictions)
print("Accuracy : {}".format(accuracy*100))

Log_loss : 0.409633875419831
Hamming_loss : 3.814031766067109
Accuracy : 90.13411170716314
